In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import random

from baseline import exhaustive, random_placement, heft, random_op_greedy_dev, random_op_est_dev
from env.utils import generate_program, generate_network
from env.network import StarNetwork
from env.program import Program
from env.latency import evaluate

from placement_rl.placement_env import PlacementEnv
from placement_rl.placement_agent import PlacementAgent

import os
import pickle

import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Using backend: pytorch


In [2]:
n_devices = [10, 20, 30]
n_operators = [10, 20, 30]
networks = {n: StarNetwork(*generate_network(n, seed=0)) for n in n_devices}
programs = {}
for n in n_devices:
    programs[n] = {}
    for m in n_operators:
        programs[n][m] = []
        for seed in range(5):
            DAG, constraints = generate_program(m, n, seed=seed)
            programs[n][m].append(Program(DAG, constraints))

In [ ]:
noises = [0, 0.01, 0.05, 0.1]
n_episodes = 20
n_iter = 50

file = open('random_op_data', 'wb')

for ndev in [10]:
    network = networks[ndev]
    for nop in n_operators:
        for seed, program in enumerate(programs[ndev][nop]):
            mapping = program.random_mapping()
            for noise in noises:
                r = {'n_devices': ndev, 
                    'n_operators': nop, 
                    'seed': seed, 
                    'noise': noise, 
                    'episodes': []}
                print(r)
                for i in range(n_episodes):
                    s = time.time()
                    random_mapping, random_latency, random_latency_trace = random_op_greedy_dev(program, network, mapping, n_iter, noise)
                    t = time.time() - s
                    r['episodes'].append([t, random_mapping, random_latency, random_latency_trace])
                    print(f'{t}  {random_mapping}')
            pickle.dump(r, file)

{'n_devices': 10, 'n_operators': 10, 'seed': 0, 'noise': 0, 'episodes': []}
1.3002028465270996  [3, 2, 5, 9, 9, 4, 4, 4, 4, 7]
1.4372971057891846  [3, 2, 6, 1, 1, 4, 9, 9, 4, 7]
1.354006052017212  [3, 2, 5, 9, 9, 4, 9, 9, 8, 7]
1.3271880149841309  [3, 2, 6, 1, 1, 4, 4, 4, 1, 7]
1.292492151260376  [3, 2, 7, 9, 9, 4, 9, 9, 9, 7]
1.3808860778808594  [3, 2, 6, 9, 9, 4, 9, 9, 1, 7]
1.3957829475402832  [3, 2, 4, 1, 1, 4, 9, 9, 8, 7]
1.2418627738952637  [3, 2, 4, 9, 9, 4, 9, 9, 1, 7]
1.2905259132385254  [3, 2, 8, 1, 1, 4, 9, 9, 9, 7]
1.3838679790496826  [3, 2, 7, 1, 1, 4, 4, 4, 4, 7]
1.3667030334472656  [3, 2, 5, 1, 1, 4, 9, 9, 8, 7]
1.2776567935943604  [3, 2, 2, 1, 1, 4, 4, 4, 1, 7]
1.3442916870117188  [3, 2, 8, 9, 9, 4, 9, 9, 9, 7]
1.7128572463989258  [3, 2, 4, 1, 1, 4, 4, 4, 4, 7]
1.1832849979400635  [3, 2, 2, 9, 3, 4, 4, 4, 4, 7]
1.3353562355041504  [3, 2, 1, 9, 9, 4, 9, 9, 1, 7]
1.2185938358306885  [3, 2, 4, 9, 9, 4, 4, 4, 1, 7]
1.235485315322876  [3, 2, 5, 1, 1, 4, 4, 4, 8, 7]
1.4287519

1.4748327732086182  [2, 9, 8, 0, 0, 6, 2, 0, 2, 6]
1.399684190750122  [2, 1, 4, 2, 1, 8, 2, 8, 2, 6]
1.2830109596252441  [2, 1, 5, 0, 0, 1, 2, 8, 2, 6]
0.9950838088989258  [2, 4, 4, 2, 0, 8, 9, 0, 2, 6]
1.0934808254241943  [2, 8, 8, 2, 8, 4, 2, 8, 2, 6]
1.0650660991668701  [2, 4, 4, 2, 1, 0, 2, 7, 2, 6]
1.0400481224060059  [2, 4, 4, 0, 0, 8, 9, 0, 2, 6]
1.1971089839935303  [2, 8, 8, 2, 8, 4, 9, 8, 2, 6]
1.2979719638824463  [2, 8, 1, 8, 8, 8, 2, 8, 2, 6]
1.114128828048706  [2, 4, 8, 0, 0, 6, 2, 0, 2, 6]
{'n_devices': 10, 'n_operators': 10, 'seed': 2, 'noise': 0, 'episodes': []}
0.7421960830688477  [5, 8, 0, 0, 2, 8, 2, 9, 2, 7]
0.715968132019043  [5, 9, 0, 0, 2, 8, 2, 9, 2, 7]
0.7374751567840576  [5, 1, 8, 8, 2, 8, 2, 1, 2, 7]
0.7888109683990479  [5, 1, 8, 8, 2, 8, 9, 2, 2, 7]
0.7764708995819092  [5, 9, 0, 0, 2, 6, 2, 1, 2, 7]
0.7707321643829346  [5, 6, 6, 6, 2, 8, 2, 1, 2, 7]
0.7553310394287109  [5, 8, 8, 8, 2, 8, 2, 1, 2, 7]
0.7332050800323486  [5, 1, 0, 0, 2, 8, 2, 1, 2, 7]
0.8062629

0.8202579021453857  [0, 0, 0, 2, 2, 4, 2, 2, 2, 4]
0.8942382335662842  [0, 0, 0, 8, 8, 1, 8, 8, 8, 4]
0.8667998313903809  [0, 0, 0, 2, 2, 1, 9, 2, 9, 4]
1.0721220970153809  [0, 6, 6, 2, 2, 4, 9, 8, 2, 4]
1.098907232284546  [0, 0, 0, 8, 8, 1, 3, 2, 3, 4]
0.999812126159668  [0, 6, 6, 8, 9, 1, 8, 8, 8, 4]
0.8782100677490234  [0, 6, 6, 2, 2, 4, 2, 2, 2, 4]
0.9094488620758057  [0, 0, 0, 2, 2, 1, 8, 8, 8, 4]
0.9777560234069824  [0, 0, 0, 2, 2, 4, 9, 2, 9, 4]
1.05377197265625  [0, 0, 1, 8, 8, 4, 3, 8, 3, 4]
0.9499471187591553  [0, 0, 6, 2, 9, 4, 9, 2, 9, 4]
0.8311138153076172  [0, 0, 0, 9, 9, 1, 9, 8, 2, 4]
0.8342890739440918  [0, 0, 0, 2, 9, 1, 9, 2, 9, 4]
0.8147881031036377  [0, 6, 6, 9, 9, 4, 9, 8, 9, 4]
0.9364140033721924  [0, 6, 6, 8, 8, 4, 8, 2, 9, 4]
1.117793083190918  [0, 1, 1, 9, 8, 4, 9, 8, 9, 4]
0.9078488349914551  [0, 0, 0, 8, 8, 4, 9, 0, 9, 4]
0.9394869804382324  [0, 0, 0, 9, 8, 1, 9, 2, 9, 4]
1.4124689102172852  [0, 1, 1, 2, 2, 4, 8, 2, 8, 4]
1.1194911003112793  [0, 6, 6, 2, 2, 

5.4982008934021  [9, 0, 9, 9, 8, 8, 5, 8, 2, 2, 2, 0, 0, 2, 0, 1, 9, 0, 0, 7]
5.106183767318726  [9, 0, 2, 2, 2, 6, 1, 6, 2, 2, 2, 0, 0, 2, 0, 1, 1, 0, 0, 7]
4.694184064865112  [9, 0, 2, 2, 2, 8, 9, 9, 2, 2, 2, 0, 0, 2, 0, 1, 5, 0, 0, 7]
4.763715744018555  [9, 0, 2, 2, 2, 8, 6, 8, 9, 9, 9, 0, 0, 2, 0, 1, 6, 0, 0, 7]
5.184638977050781  [9, 0, 2, 2, 2, 6, 4, 6, 2, 2, 2, 0, 0, 2, 0, 1, 7, 0, 0, 7]
3.9305059909820557  [9, 0, 2, 2, 2, 8, 5, 8, 9, 9, 9, 0, 0, 2, 0, 1, 9, 0, 0, 7]
{'n_devices': 10, 'n_operators': 20, 'seed': 0, 'noise': 0.05, 'episodes': []}
4.1075520515441895  [9, 0, 2, 2, 3, 8, 5, 4, 9, 8, 1, 1, 0, 2, 3, 1, 4, 1, 3, 7]
4.336601972579956  [9, 3, 9, 9, 8, 8, 1, 2, 2, 2, 2, 0, 0, 2, 0, 1, 4, 0, 0, 7]
5.788283109664917  [9, 0, 9, 2, 2, 8, 1, 9, 9, 2, 2, 0, 0, 2, 0, 1, 7, 0, 0, 7]
5.208262920379639  [9, 3, 2, 2, 2, 8, 7, 8, 9, 5, 2, 0, 0, 2, 0, 1, 4, 0, 0, 7]
5.557563781738281  [9, 0, 8, 8, 8, 6, 7, 6, 2, 2, 2, 0, 0, 2, 0, 1, 4, 0, 0, 7]
6.247744083404541  [9, 0, 2, 2, 8, 8, 7, 

6.372749090194702  [6, 2, 2, 2, 2, 8, 4, 9, 7, 3, 9, 6, 6, 6, 8, 2, 3, 2, 8, 1]
7.047141075134277  [6, 3, 3, 3, 3, 1, 3, 1, 8, 3, 9, 7, 7, 7, 4, 2, 9, 3, 8, 1]
7.435929775238037  [6, 2, 3, 3, 3, 4, 3, 8, 8, 9, 9, 0, 0, 8, 0, 9, 2, 2, 6, 1]
7.318548917770386  [6, 2, 2, 2, 2, 8, 2, 9, 7, 3, 6, 6, 9, 8, 7, 2, 9, 3, 1, 1]
6.4203619956970215  [6, 2, 2, 2, 2, 8, 2, 1, 1, 3, 9, 0, 2, 9, 1, 9, 2, 3, 4, 1]
6.100023031234741  [6, 2, 2, 2, 2, 6, 3, 8, 7, 2, 9, 0, 8, 9, 8, 9, 2, 2, 8, 1]
7.14125394821167  [6, 3, 3, 3, 3, 8, 2, 2, 2, 3, 9, 0, 2, 9, 1, 9, 2, 3, 6, 1]
7.13048529624939  [6, 2, 2, 2, 9, 1, 2, 2, 8, 2, 6, 7, 2, 9, 8, 9, 2, 3, 1, 1]
{'n_devices': 10, 'n_operators': 20, 'seed': 1, 'noise': 0.1, 'episodes': []}
7.171757221221924  [6, 3, 3, 3, 4, 8, 2, 2, 7, 2, 9, 7, 2, 6, 4, 9, 2, 2, 6, 1]
7.346040964126587  [6, 2, 2, 2, 2, 8, 2, 1, 2, 2, 9, 0, 2, 8, 1, 2, 9, 2, 1, 1]
9.040618896484375  [6, 2, 3, 3, 3, 8, 2, 8, 1, 3, 9, 7, 9, 8, 4, 2, 9, 3, 4, 1]
7.000356912612915  [6, 2, 9, 4, 4, 1, 4, 2,

In [ ]:
for ndev in [20, 30]:
    network = networks[ndev]
    for nop in n_operators:
        for seed, program in enumerate(programs[ndev][nop]):
            mapping = program.random_mapping()
            for noise in noises:
                r = {'n_devices': ndev, 
                    'n_operators': nop, 
                    'seed': seed, 
                    'noise': noise, 
                    'episodes': []}
                print(r)
                for i in range(n_episodes):
                    s = time.time()
                    random_mapping, random_latency, random_latency_trace = random_op_greedy_dev(program, network, mapping, n_iter, noise)
                    t = time.time() - s
                    r['episodes'].append([t, random_mapping, random_latency, random_latency_trace])
                    print(f'{t}  {random_mapping}')
            pickle.dump(r, file)